# 利用Keras快速构建和训练四种CNN结构
本文利用Keras分别对**单输入单输出、单输入多输出、多输入单输出、多输入多输出**四种结构下的CNN网络结构进行快速构建和验证，并分别在普通模式和生成器模式下对所构建的模型进行训练和预测。实际应用中无外乎是这四种结构的变形，因此本文中构建和训练模型的方法可以作为最小原型参考，以便读者构建和训练对具体任务中所要构建的复杂网络。<br>
实际任务中可能更多的需要根据特定的数据集构建**生成器**，本文提供的几种生成器的构建方法也有较强的参考意义。

In [125]:
import keras
from keras.layers import Input,Conv2D,MaxPooling2D,Dense,Flatten,concatenate
from keras.models import Sequential,Model
import numpy as np
from keras.optimizers import SGD

In [169]:
# 本文涉及到的输入输出数据
x_train = np.random.random((64,32,32,3))
x_train2 = np.random.random((64,32,32,3))
y_train = keras.utils.to_categorical(np.random.randint(10,size=(64,1)), num_classes=10)
y_train2 = keras.utils.to_categorical(np.random.randint(5,size=(64,1)),num_classes=5)
x_train.shape,y_train.shape,x_train2.shape,y_train2.shape

((64, 32, 32, 3), (64, 10), (64, 32, 32, 3), (64, 5))

In [168]:
# 定义一个全局使用的optimizer
optimizer = SGD(lr = 0.001, momentum = 0.9, decay = 0.0, nesterov = True)

## 1 单输入单输出模型

### 1.1 单入单出普通训练模式

In [170]:
# 由于比较单入单出模型较简单，这里用Sequential方式构建
model1 = Sequential([
    Conv2D(16,(3,3),padding='same',activation='relu', input_shape=(32,32,3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32,(3,3),padding='same',activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(64,activation='relu'),
    Dense(10,activation='softmax')]
)
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_37 (Dense)             (None, 64)                131136    
_________________________________________________________________
dense_38 (Dense)             (None, 10)                650       
Total para

In [171]:
model1.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model1.fit(x_train,y_train,2)
score1 = model1.predict(x_train)
score1.shape

Epoch 1/1
64/64 [==============================] - 1s 21ms/step - loss: 2.3625 - acc: 0.1094


(64, 10)

### 1.2 单入单出生成器模式

In [133]:
def sig_generator(x_train,y_train=None,batchsize=16,return_label=False):
    N = x_train.shape[0]
    batch_index = 0
    while True:
        cur_index = batch_index * batchsize
        if N >= (cur_index + batchsize):
            cur_batch_size = batchsize
            batch_index += 1
        else:
            cur_batch_size = N - cur_index
            batch_index = 0
        
        X_batch = x_train[cur_index:cur_index+cur_batch_size,:]
        if return_label:
            y_batch = y_train[cur_index:cur_index+cur_batch_size,:]
            yield (X_batch,y_batch)
        else:
            yield X_batch

In [174]:
batch_gen1 = sig_generator(x_train,y_train,2,True)
model1.fit_generator(batch_gen1,32)
batch_gen1_test = sig_generator(x_train) #注意这里的batchsize没必要跟训练时一致
score1_gen = model1.predict_generator(batch_gen1_test,4) 
score1_gen.shape

Epoch 1/1
32/32 [==============================] - 1s 18ms/step - loss: 2.2988 - acc: 0.1250


(64, 10)

## 2 单输入多输出模型

### 2.1 单入多出普通训练模式

In [175]:
# 利用函数式构建
inputs = Input(shape=(32,32,3))
conv_layer1 = Conv2D(16,(3,3),padding='same',activation='relu')(inputs)
max_layer1 = MaxPooling2D(pool_size=(2,2))(conv_layer1)
conv_layer2 = Conv2D(32,(3,3),padding='same',activation='relu')(max_layer1)
max_layer2 = MaxPooling2D(pool_size=(2,2))(conv_layer2)
flatten = Flatten()(max_layer2)
dence1 = Dense(64,activation='relu')(flatten)
f1 = Dense(10, activation='softmax', name='prediction_one')(dence1)
f2 = Dense(5, activation='softmax', name='prediction_two')(dence1)
model2 = Model(outputs=[f1,f2], inputs=inputs)
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 32, 32, 16)   448         input_31[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_51 (MaxPooling2D) (None, 16, 16, 16)   0           conv2d_51[0][0]                  
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 16, 16, 32)   4640        max_pooling2d_51[0][0]           
__________________________________________________________________________________________________
max_poolin

In [188]:
# 注意这里的loss是一个list,还可以指定loss_weight进行加权
model2.compile(loss=['categorical_crossentropy','categorical_crossentropy'],optimizer=optimizer,metrics=['accuracy'])
model2.fit(x_train,[y_train,y_train2],2)
score2 = model2.predict(x_train)
# 多输出的结果为一个list
for score in score2:
    print(score.shape)

Epoch 1/1
64/64 [==============================] - 2s 26ms/step - loss: 3.8765 - prediction_one_loss: 2.2725 - prediction_two_loss: 1.6041 - prediction_one_acc: 0.1250 - prediction_two_acc: 0.2188
(64, 10)
(64, 5)


### 2.2 单入多出生成器模式

In [141]:
def dul_generator(x_train,y_train=None,y_train2=None,batchsize=16,return_label=False):
    N = x_train.shape[0]
    batch_index = 0
    while True:
        cur_index = batch_index * batchsize
        if N >= (cur_index + batchsize):
            cur_batch_size = batchsize
            batch_index += 1
        else:
            cur_batch_size = N - cur_index
            batch_index = 0
        
        X_batch = x_train[cur_index:cur_index+cur_batch_size,:]
        if return_label:
            y_batch1 = y_train[cur_index:cur_index+cur_batch_size,:]
            y_batch2 = y_train2[cur_index:cur_index+cur_batch_size,:]
            yield (X_batch,[y_batch1,y_batch2])
        else:
            yield X_batch

In [177]:
batch_gen2 = dul_generator(x_train,y_train,y_train2,2,True)
model2.fit_generator(batch_gen2,32)
batch_gen2_test = dul_generator(x_train)
score2_gen = model2.predict_generator(batch_gen2_test,4)
for score in score2_gen:
    print(score.shape)

Epoch 1/1
32/32 [==============================] - 1s 19ms/step - loss: 3.8985 - prediction_one_loss: 2.2806 - prediction_two_loss: 1.6180 - prediction_one_acc: 0.1250 - prediction_two_acc: 0.2188
(64, 10)
(64, 5)


### 2.3 单输入多分支单输出模式

In [178]:
# 实际中单输入多输出更多用于提取隐含层特征，所以到不了最后的输出，比如这里在flatten_4后边跟两个dense输出不同维度的特征，然后可能做一个整合
# 这是就需要对模型做如下修改
inputs = Input(shape=(32,32,3))
conv_layer1 = Conv2D(16,(3,3),padding='same',activation='relu')(inputs)
max_layer1 = MaxPooling2D(pool_size=(2,2))(conv_layer1)
conv_layer2 = Conv2D(32,(3,3),padding='same',activation='relu')(max_layer1)
max_layer2 = MaxPooling2D(pool_size=(2,2))(conv_layer2)
flatten = Flatten()(max_layer2)
dence1 = Dense(64,activation='relu')(flatten)
dence2 = Dense(128,activation='relu')(flatten)
dence_cat = concatenate([dence1,dence2])
f1 = Dense(10, activation='softmax', name='prediction_one')(dence_cat)
model3 = Model(outputs=f1,inputs=inputs)
model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 16)   448         input_32[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_53 (MaxPooling2D) (None, 16, 16, 16)   0           conv2d_53[0][0]                  
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 16, 16, 32)   4640        max_pooling2d_53[0][0]           
__________________________________________________________________________________________________
max_poolin

In [180]:
# 其实仍是一个单输入单输出模型，只不过内部结构复杂了一些
model3.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model3.fit(x_train,y_train,2)
score3 = model3.predict(x_train)
score3.shape

Epoch 1/1
64/64 [==============================] - 2s 28ms/step - loss: 2.4516 - acc: 0.0469


(64, 10)

In [182]:
batch_gen3 = sig_generator(x_train,y_train,2,True)
model3.fit_generator(batch_gen3,32)
batch_gen3_test = sig_generator(x_train)
score3_gen = model3.predict_generator(batch_gen3_test,4)
score3_gen.shape

Epoch 1/1
32/32 [==============================] - 1s 27ms/step - loss: 2.2951 - acc: 0.1094


(64, 10)

## 3 多输入单输出模型

### 3.1 多入单出普通训练模式

In [183]:
# 输入可以在任意层进行的整合，这里我们只对输入层进行整合
input1 = Input(shape=(32,32,3))
input2 = Input(shape=(32,32,3))
cc = concatenate([input1, input2])
conv_layer1 = Conv2D(16,(3,3),padding='same',activation='relu')(cc)
max_layer1 = MaxPooling2D(pool_size=(2,2))(conv_layer1)
conv_layer2 = Conv2D(32,(3,3),padding='same',activation='relu')(max_layer1)
max_layer2 = MaxPooling2D(pool_size=(2,2))(conv_layer2)
flatten = Flatten()(max_layer2)
dence1 = Dense(64,activation='relu')(flatten)
f1 = Dense(10, activation='softmax', name='prediction_one')(dence1)
model4 = Model(outputs=f1,inputs=[input1,input2])
model4.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
concatenate_19 (Concatenate)    (None, 32, 32, 6)    0           input_33[0][0]                   
                                                                 input_34[0][0]                   
__________________________________________________________________________________________________
conv2d_55 (Conv2D)              (None, 32, 32, 16)   880         concatenate_19[0][0]             
__________

In [184]:
model4.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model4.fit([x_train,x_train2],y_train,2)
score4 = model4.predict([x_train,x_train2])
score4.shape

Epoch 1/1
64/64 [==============================] - 1s 22ms/step - loss: 2.3626 - acc: 0.1250


(64, 10)

### 3.2 多入单出生成器模式

In [185]:
# 要求x_train和x_train2有同样的维度
def dul_generator2(x_train,x_train2,y_train=None,batchsize=16,return_label=False):
    N = x_train.shape[0]
    batch_index = 0
    while True:
        cur_index = batch_index * batchsize
        if N >= (cur_index + batchsize):
            cur_batch_size = batchsize
            batch_index += 1
        else:
            cur_batch_size = N - cur_index
            batch_index = 0
        
        X_batch1 = x_train[cur_index:cur_index+cur_batch_size,:]
        X_batch2 = x_train2[cur_index:cur_index+cur_batch_size,:]
        if return_label:
            y_batch = y_train[cur_index:cur_index+cur_batch_size,:]
            yield ([X_batch1,X_batch2],y_batch)
        else:
            yield [X_batch1,X_batch2]

In [186]:
batch_gen4 = dul_generator2(x_train,x_train2,y_train,2,True)
model4.fit_generator(batch_gen4,32)
batch_gen4_test = dul_generator2(x_train,x_train2)
score4 = model4.predict_generator(batch_gen4_test,4)
score4.shape

Epoch 1/1
32/32 [==============================] - 1s 16ms/step - loss: 2.2944 - acc: 0.0938


(64, 10)

## 4 多输入多输出模型
将单输入多输出和多输入单输出结合起来

### 4.1 多入多出普通训练模式

In [189]:
input1 = Input(shape=(32,32,3))
input2 = Input(shape=(32,32,3))
cc = concatenate([input1, input2])
conv_layer1 = Conv2D(16,(3,3),padding='same',activation='relu')(cc)
max_layer1 = MaxPooling2D(pool_size=(2,2))(conv_layer1)
conv_layer2 = Conv2D(32,(3,3),padding='same',activation='relu')(max_layer1)
max_layer2 = MaxPooling2D(pool_size=(2,2))(conv_layer2)
flatten = Flatten()(max_layer2)
dence1 = Dense(64,activation='relu')(flatten)
f1 = Dense(10, activation='softmax', name='prediction_one')(dence1)
f2 = Dense(5, activation='softmax', name='prediction_two')(dence1)
model5 = Model(outputs=[f1,f2],inputs=[input1,input2])
model5.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
concatenate_21 (Concatenate)    (None, 32, 32, 6)    0           input_37[0][0]                   
                                                                 input_38[0][0]                   
__________________________________________________________________________________________________
conv2d_59 (Conv2D)              (None, 32, 32, 16)   880         concatenate_21[0][0]             
__________

In [190]:
model5.compile(loss=['categorical_crossentropy','categorical_crossentropy'],optimizer=optimizer,metrics=['accuracy'])
model5.fit([x_train,x_train2],[y_train,y_train2],2)
score5 = model5.predict([x_train,x_train2])
for score in score5:
    print(score.shape)

Epoch 1/1
64/64 [==============================] - 2s 38ms/step - loss: 4.0508 - prediction_one_loss: 2.3733 - prediction_two_loss: 1.6775 - prediction_one_acc: 0.0625 - prediction_two_acc: 0.2500
(64, 10)
(64, 5)


### 4.2 多入多出生成器模式

In [162]:
def dual_generator(x_train,x_train2,y_train=None,y_train2=None,batchsize=16,return_label=False):
    N = x_train.shape[0]
    batch_index = 0
    while True:
        cur_index = batch_index * batchsize
        if N >= (cur_index + batchsize):
            cur_batch_size = batchsize
            batch_index += 1
        else:
            cur_batch_size = N - cur_index
            batch_index = 0
        
        X_batch1 = x_train[cur_index:cur_index+cur_batch_size,:]
        X_batch2 = x_train2[cur_index:cur_index+cur_batch_size,:]
        if return_label:
            y_batch1 = y_train[cur_index:cur_index+cur_batch_size,:]
            y_batch2 = y_train2[cur_index:cur_index+cur_batch_size,:]
            yield ([X_batch1,X_batch2],[y_batch1,y_batch2])
        else:
            yield [X_batch1,X_batch2]

In [191]:
batch_gen5 = dual_generator(x_train,x_train2,y_train,y_train2,2,True)
model5.fit_generator(batch_gen5,32)
batch_gen5_test = dual_generator(x_train,x_train2)
score5 = model5.predict_generator(batch_gen5_test,4)
for score in score5:
    print(score.shape)

Epoch 1/1
32/32 [==============================] - 1s 20ms/step - loss: 3.8681 - prediction_one_loss: 2.2751 - prediction_two_loss: 1.5930 - prediction_one_acc: 0.1875 - prediction_two_acc: 0.2500
(64, 10)
(64, 5)
